In [ ]:
# 1. IMPORTAR MÓDULOS
from extractors import (
    MySqlSourceExtractor, OracleExtractor, PostgresExtractor, 
    SqlServerExtractor, SqliteExtractor
)
from transformers import standardize_columns, clean_and_cast_data
from loaders import MySqlLoader
import pandas as pd

# Definición de la query genérica para ventas (debes ajustarla a cada BD)
VENTAS_QUERY_MYSQL = "SELECT * FROM ventas_table" 

# Lista de extractores y nombres de fuente (para auditoría)
extractor_map = {
    "MySQL_Source": MySqlSourceExtractor(),
    "Oracle_Source": OracleExtractor(),
    "Postgres_Source": PostgresExtractor(),
    "SQLServer_Source": SqlServerExtractor(),
    "SQLite_Source": SqliteExtractor()
}

all_consolidated_dfs = []

In [ ]:
# 2. PROCESAMIENTO ETL ITERATIVO (OCP)
print("--- INICIO DEL PIPELINE DE VENTAS (E X T R A C C I Ó N) ---")

for source_name, extractor in extractor_map.items():
    print(f"\n[PROCESANDO FUENTE: {source_name}]")
    
    # E: Extracción (LSP: Se garantiza que cada extractor tiene fetch_data)
    df_raw = extractor.fetch_data(VENTAS_QUERY)
    
    if df_raw.empty:
        continue # Si falla la extracción, pasa a la siguiente fuente

    # T: Transformación y Estandarización
    df_transformed = standardize_columns(df_raw.copy(), source_name)
    df_cleaned = clean_and_cast_data(df_transformed)
    
    # Asignar la fuente para auditoría (Lógica específica del Notebook)
    df_cleaned['source_db'] = source_name
    
    # Almacenar para consolidación
    all_consolidated_dfs.append(df_cleaned)
    print(f"-> DataFrame de {source_name} listo para consolidación.")

In [ ]:
# 3. CONSOLIDACIÓN
if not all_consolidated_dfs:
    print("\nFATAL: No se extrajeron datos de ninguna fuente. Terminando.")
else:
    print("\n--- INICIO DE CONSOLIDACIÓN ---")
    df_final = pd.concat(all_consolidated_dfs, ignore_index=True, sort=False)
    
    # T: Transformación final (ej. manejar valores que faltan tras consolidación)
    df_final.fillna(value={'amount': 0.0, 'customer_id': 'N/A'}, inplace=True)
    
    print(f"Total de registros a cargar: {len(df_final)}")
    print("Columnas finales:", df_final.columns.tolist())
    print(df_final.head())
    
    # 4. CARGA (L)
    print("\n--- INICIO DE CARGA (L O A D) ---")
    mysql_loader = MySqlLoader() # DIP: El notebook usa el Loader sin saber los detalles de conexión
    mysql_loader.load_data(df_final, table_name="ventas_consolidado")
    
    print("\n>>> PIPELINE ETL FINALIZADO EXITOSAMENTE <<<")